In [1]:
import sys
sys.path.extend(["../", "../src", "../config"])

In [2]:
import pandas as pd
from model_factory import SpacyModel

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\suvro\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
df = pd.read_json("../data.json")

In [4]:
model = SpacyModel(data=df, model_name='en_core_web_lg')

In [5]:
model.performance_metrics()

100%|█████████████████████████████████████████████████████████████████████████| 1000/1000 [01:07<00:00, 14.91it/s]


Classification report for en_core_web_lg:
               precision    recall  f1-score   support

Senior Level       0.65      0.85      0.73        13
 Entry Level       0.50      1.00      0.67         3
  Internship       1.00      1.00      1.00         3
   Mid Level       1.00      0.30      0.46        10

    accuracy                           0.69        29
   macro avg       0.79      0.79      0.72        29
weighted avg       0.79      0.69      0.66        29



In [6]:
model.initialize_docbins(generate_test_file=True)

In [7]:
# Train transformer model
model.train_cli()

ℹ Saving to output directory: ..\trained_pipelines
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'textcat_multilabel']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS TEXTC...  CATS_SCORE  SCORE 
---  ------  -------------  -------------  ----------  ------
  0       0           0.10           0.19       53.26    0.53
  1     100           6.63          31.04       62.14    0.62
  3     200           4.30          23.74       62.27    0.62
  5     300           3.65          21.59       67.10    0.67
  7     400           2.90          20.31       71.11    0.71
  8     500           2.71          19.91       73.00    0.73
 10     600           3.17          17.80       75.74    0.76
 12     700           2.62          16.86       76.53    0.77
 14     800           2.85          14.40       83.56    0.84
 16     900           1.92          13.43       78.82    0.79
 17    1000           1.99          11.19       84.05    0.84
 19    1100     

In [8]:
model.eval_cli()

ℹ Using GPU: 0

================================== Results ==================================

TOK                   100.00
TEXTCAT (macro AUC)   92.62 
SPEED                 6463  


=========================== Textcat F (per label) ===========================

                   P        R       F
Entry Level    50.00    66.67   57.14
Senior Level   84.62    84.62   84.62
Internship     75.00   100.00   85.71
Mid Level      87.50    70.00   77.78


======================== Textcat ROC AUC (per label) ========================

               ROC AUC
Entry Level       0.95
Senior Level      0.85
Internship        1.00
Mid Level         0.91



In [9]:
# Apply model on test/validation data
!python -m spacy apply ..\trained_pipelines\model-best\ ..\data\val.spacy ..\outputs\test_pred.spacy --force --gpu-id 0

[i] Using GPU: 0
[+] Loaded model ..\trained_pipelines\model-best\



0it [00:00, ?it/s]
1it [00:01,  1.50s/it]
6it [00:01,  4.92it/s]
10it [00:01,  8.68it/s]
14it [00:01, 12.48it/s]
19it [00:01, 17.69it/s]
25it [00:02, 24.65it/s]
29it [00:02, 13.45it/s]


In [10]:
model.performance_metrics(from_saved_model=True)

Classification report for transformer roberta-base:
               precision    recall  f1-score   support

 Entry Level       0.50      0.67      0.57         3
   Mid Level       0.88      0.70      0.78        10
  Internship       0.75      1.00      0.86         3
Senior Level       0.85      0.85      0.85        13

    accuracy                           0.79        29
   macro avg       0.74      0.80      0.76        29
weighted avg       0.81      0.79      0.80        29



In [11]:
# Apply model on actual test data
!python -m spacy apply ..\trained_pipelines\model-best\ ..\data\test.spacy ..\outputs\test_pred.spacy --force --gpu-id 0

[i] Using GPU: 0
[+] Loaded model ..\trained_pipelines\model-best\



0it [00:00, ?it/s]
1it [00:01,  1.51s/it]
5it [00:01,  4.01it/s]
9it [00:01,  7.87it/s]
14it [00:01, 13.30it/s]
18it [00:01, 17.40it/s]
23it [00:02, 22.32it/s]
28it [00:02, 26.97it/s]
33it [00:02, 30.50it/s]
38it [00:02, 33.23it/s]
43it [00:02, 34.23it/s]
49it [00:02, 37.28it/s]
54it [00:02, 38.47it/s]
59it [00:02, 38.77it/s]
64it [00:03, 38.98it/s]
69it [00:03, 39.83it/s]
74it [00:03, 40.55it/s]
75it [00:03, 22.46it/s]


In [12]:
test_pred_levels = model.get_test_preds()

In [13]:
test = model.test_df.copy(deep=True)
test.drop('desc_title', axis=1, inplace=True)
test['level'] = test_pred_levels

In [14]:
test

,level,description,title
141,Entry Level,OUTFITTERY is Europe’s biggest Personal Shoppi...,Customer Service Netherlands in Berlin (m/f)
142,Senior Level,OUTFITTERY is Europe’s biggest Personal Shoppi...,DevOps Engineer (m/f)
143,Senior Level,OUTFITTERY is Europe’s biggest Personal Shoppi...,Head of Product Management (IT) (m/f)
144,Entry Level,OUTFITTERY is Europe’s biggest Personal Shoppi...,Help Desk Support (m/f)
145,Internship,OUTFITTERY is Europe’s biggest Personal Shoppi...,Intern Help Desk (m/f)
...,...,...,...
211,Entry Level,Work in a one of a kind international environm...,Native Italian Translation Internship
212,Internship,Work in a unique international environment and...,International Translation Internship Program (...
213,Internship,Photography - New York Habitat Internship\nWho...,Photography - New York Habitat Internship
214,Entry Level,Are you free every day from 10:30am - 1:30pm a...,High Paying Part Time Delivery Driver


In [15]:
test.to_json("../outputs/test_pred.json", orient='records')